# ipyleaflet

In [1]:
from __future__ import print_function
from ipyleaflet import *

In [2]:
m = Map(center=(48, 8), zoom=8, basemap=basemaps.Hydda.Full)
m

Map(center=[48, 8], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text…

In [3]:
strata_all = basemap_to_tiles(basemaps.Strava.All)
m.add_layer(strata_all)

In [4]:
from ipywidgets import Label

label = Label()
display(label)

def handle_interaction(**kwargs):
    if kwargs.get('type') == 'mousemove':
        label.value = str(kwargs.get('coordinates'))

m.on_interaction(handle_interaction)

Label(value='')

In [5]:
strata_water = basemap_to_tiles(basemaps.Strava.Water)
m.substitute_layer(strata_all, strata_water)

In [6]:
import json
from geojson import GeoJSON

with open('./europe_110.geo.json') as f:
    data = json.load(f)
for feature in data['features']:
    feature['properties']['style'] = {
        'color': 'grey',
        'weight': 1,
        'fillColor': 'grey',
        'fillOpacity': 0.5
    }
geo = GeoJSON(data=data, hover_style={'fillColor': 'red'}, name='Countries')
m.add_layer(geo)

In [7]:
m.add_control(LayersControl())

In [8]:
import ipywidgets
 
ipywidgets.HBox([m, Map(center=[48.0, 7.9], zoom=10)])

## WMS-Layer

Mit ipyleaflet lassen sich auch eigene [WMS](https://de.wikipedia.org/wiki/Web_Map_Service)-Layer definiern. Üblicherweise werden Optionen wie `layers`, `format` und `transparent` in der Request-URL übergeben. Wenn ihr  zusätzliche Parameter benötigt, könnt ihr diese jedoch auch in eigenen WMSLayer-Klassen definieren. Im folgenden wird beispielsweise ein Zeitparameter hinzugefügt, indem ein benutzerdefinierter `TimeWMSLayer` definiert wird.

### Beispiel für einen eigenen WMS-Layer

In [9]:
from ipyleaflet import Map, WMSLayer, basemaps
from traitlets import Unicode

class TimeWMSLayer(WMSLayer):
    time = Unicode('').tag(sync=True, o=True)

time_wms = TimeWMSLayer(
    url='https://mesonet.agron.iastate.edu/cgi-bin/wms/nexrad/n0r-t.cgi?',
    layers='nexrad-n0r-wmst',
    time='2005-08-29T13:00:00Z',
    format='image/png',
    transparent=True,
    attribution='Weather data © 2012 IEM Nexrad'
)

m = Map(basemap=basemaps.CartoDB.Positron, center=(30.661, -88.645), zoom=5)

m.add_layer(time_wms)

m

Map(center=[30.661, -88.645], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

Da es sich um ein Widget handelt, könnt ihr den WMS-Parameter `time_wms.time` auch einfachändern:

In [10]:
time_wms.time = '2005-08-29T19:00'

Oder ihr könnt einen Slider mit passenden Zeitintervallen definieren:

In [11]:
from ipywidgets import SelectionSlider

time_options = [
    '13:00', '13:30',
    '14:00', '14:30',
    '15:00', '15:30',
    '16:00', '16:30'
]

slider = SelectionSlider(description='Time:', options=time_options)

def update_wms(change):
    time_wms.time = '2005-08-29T{}'.format(slider.value)

slider.observe(update_wms, 'value')

slider

SelectionSlider(description='Time:', options=('13:00', '13:30', '14:00', '14:30', '15:00', '15:30', '16:00', '…

### WMS-Layer-Attribute

| Attribute | Standardwert | Kommentar |
| :-------- | :----------- | :-------- |
| `url` | `"https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png"` |  |
| `min_zoom` | `0` |  |
| `max_zoom` | `18` |  |
| `tile_size` | `256` |  |
| `attribution` | `"Map data (c) <a href='https://openstreetmap.org'>OpenStreetMap</a> contributors"` |  |
| `detect_retina` | `False` |  |
| `opacity` | `1.0` |  |
| `visible` | `True` |  |
| `service` | `"WMS"` |  |
| `request` | `"GetMap"` |  |
| `layers` | `""` | Kommagetrennte Liste der anzuzeigenden WMS-Ebenen |
| `styles` | `""` | Kommagetrennte Liste der WMS-Stile |
| `format` | `"image/jpeg"` | Für Ebenen mit Transparenz solltet ihr `"image/png"` verwenden |
| `transparent` | `False` |  |
| `version` | `"1.1.1"` | Version des zu verwendenden WMS-Dienstes |
| `crs` | `""` |  |


## Siehe auch

* [ipyleaflet: Interactive maps in the Jupyter notebook](https://ipyleaflet.readthedocs.io/en/latest/)